In [3]:
import torch, torchvision
from tqdm import tqdm

In [2]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=64+32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.elu = torch.nn.ELU()
        self.mp = torch.nn.MaxPool2d(kernel_size=2, stride=2)
            
        self.lin_layers = torch.nn.Sequential(
            torch.nn.Linear(in_features=20*7*7, out_features=32),
            torch.nn.ELU(),
            torch.nn.Linear(in_features=32, out_features=10),
            torch.nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.elu(self.conv1(x))
        l1Out = self.mp(x)
        l20ut = self.elu(self.conv2(l1Out))
        l2Out = torch.nn.functional.interpolate(l2Out, size=(l2Out.size[0],l2Out.size[1],l1Out.size[2], l1Out.size[3]))
        x = torch.cat([l1Out, l2Out], dim=1)
        x = self.elu(self.conv3(x))
        x = self.mp(x)
        x = x.view(-1, 64*7*7)
        x = self.lin_layers(x)
        return x

NameError: name 'torch' is not defined

In [56]:
model = ConvNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
criterion = torch.nn.CrossEntropyLoss()

In [57]:
xforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])  # Try with no normalization for now
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=xforms)
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=xforms)
batch_size=64
train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=True)

In [58]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
writer.add_image('images', grid, 0)

In [59]:
epochs = 15
loss_history = []
for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    val_loss = 0
    for data, labels in train_loader:

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    writer.add_scalar(tag="loss", scalar_value=epoch_loss/len(train_loader), global_step=epoch)
    
writer.close()

100%|██████████| 15/15 [17:49<00:00, 80.94s/it]


In [60]:
correct_count = 0
model.eval()
with torch.no_grad():
    for image, label in test_loader:
        output = model(image)
        
        pred = output.argmax(dim=1, keepdim=True)
        correct_count += pred.eq(label.view_as(pred)).sum().item()
        
print(f"Correct/Total: {correct_count}/{len(test_loader.dataset)}")
print(f"Accuracy: {correct_count/len(test_loader.dataset):.4f}")

Correct/Total: 8834/10000
Accuracy: 0.8834
